In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader

# ResNet50 모델 불러오기
model = models.resnet50(pretrained=True)

# 마지막 fully connected layer 수정
num_ftrs = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(num_ftrs, 1),  # 1개의 출력 (0- 1)
    nn.Sigmoid()             # Sigmoid 활성화 함수 추가
)

model = torch.load('model_v4.pt')  # 모델 불러오기

C:\Users\lwj01\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\lwj01\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
import os
from PIL import Image
from torchvision import transforms
import torch

directory = r'C:\Users\lwj01\HowFastTennisBallIs\novak_sinner_over_30'
save_directory = r'C:\Users\lwj01\HowFastTennisBallIs\y_v4'  # 저장할 경로

# CUDA 사용 가능 여부 확인 및 모델을 GPU로 이동
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
model = model.to(device)

def sliding_window(image, step_size, window_size):
    """이미지 위에서 슬라이딩 윈도우를 생성하는 함수"""
    for y in range(0, image.size[1] - window_size[1], step_size):
        for x in range(0, image.size[0] - window_size[0], step_size):
            yield (x, y, image.crop((x, y, x + window_size[0], y + window_size[1])))

def detect_objects_per_image(model, image_path, window_size=(30, 30), step_size=10):
    """하나의 이미지에 대해 객체를 탐지하는 함수"""
    model.eval()  # 모델을 평가 모드로 설정
    transform = transforms.Compose([transforms.Resize(window_size), transforms.ToTensor()])

    best_score = 0
    best_window = None
    best_location = None

    image = Image.open(image_path).convert('RGB')
    for (x, y, window) in sliding_window(image, step_size, window_size):
        window_tensor = transform(window).unsqueeze(0).to(device)  # 모델에 입력하기 위해 차원 추가
        output = model(window_tensor)
        
        score = torch.sigmoid(output[0][0]).item()
        if score > best_score:
            best_score = score
            best_window = window
            best_location = (x, y)

    return best_score, best_location, best_window

# 모든 이미지에 대해 객체 탐지 실행
for filename in os.listdir(directory):
    if filename.endswith('.jpg'):
        image_path = os.path.join(directory, filename)
        score, location, window = detect_objects_per_image(model, image_path)
        print(f"Image: {filename}, Best Score: {score}, Location: {location}")
        if window is not None:
            save_path = os.path.join(save_directory, f"detected_{filename}")
            window.save(save_path)  # 윈도우 이미지를 지정된 경로에 저장


Using device: cuda:0
Image: frame_000000.jpg, Best Score: 0.7310585379600525, Location: (0, 0)
Image: frame_000001.jpg, Best Score: 0.7310585379600525, Location: (0, 0)
Image: frame_000002.jpg, Best Score: 0.7310585379600525, Location: (0, 0)
Image: frame_000003.jpg, Best Score: 0.7310585379600525, Location: (0, 0)
Image: frame_000004.jpg, Best Score: 0.7310585379600525, Location: (0, 0)
Image: frame_000005.jpg, Best Score: 0.7310585379600525, Location: (0, 0)
Image: frame_000006.jpg, Best Score: 0.7310585379600525, Location: (0, 0)
Image: frame_000007.jpg, Best Score: 0.7310585379600525, Location: (0, 0)
Image: frame_000008.jpg, Best Score: 0.7310585379600525, Location: (0, 0)
